## Introduction:

Main Goals:

1. Identify the recipients that will engage with the campaign.
2. Maximise the campaign’s revenue.


Comments

- The dataset contains only 5% of donors.
- The donations are usually smaller than $20.
- This data is quite noisy, high dimensional.
- There is an inverse relationship between the probability to donate and the amount donated.


Link for dataset and some analysis ==> 

https://github.com/rebordao/kdd98cup

https://github.com/bobbyantonio/KDD98/blob/master/CleanData.py

- Github solutions ==>
https://github.com/rebordao/kdd98cup


- Siraj notebook for a better data visualization:

https://www.youtube.com/watch?v=yQsOFWqpjkE

- Implementations:

https://github.com/sisl/CustomerSim/blob/master/src/kdd98_preprocess.R

https://github.com/ugo-nama-kun/DQN-chainer/blob/master/dqn_agent_nature.py

- Yegor Tk's website => http://yegortkachenko.com/

In [37]:
import tensorflow as tf
import numpy as np
# from tensorflow.python.ops import rnn, rnn_cell
from sklearn.model_selection import train_test_split
import pandas as pd
import random

## plotting .. 
# import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

from collections import Counter

import random 
# random.seed(123)

import csv
import sys

## warnings
import warnings
warnings.filterwarnings("ignore")

## Loading the dataset:

In [38]:
def load_data():
    df = pd.read_csv('tuple.csv', header = 0)
    return df

In [39]:
def df_split():
    df = load_data()
    train, test = train_test_split(df, test_size = 0.9)  # split data to 50-50 cross validate 
    
    return train, test

In [40]:
# train, test = df_split()
# np.shape(train)
# len(train)

In [41]:
# np.shape(train)[1]

In [42]:
def tuple_():
    
    train, test = df_split()

    train = train.convert_objects(convert_numeric=True)
    
    next_actions = np.zeros([len(train), 12])
    
    #     # fill in next_actions  
    for i in xrange(np.shape(train)[1]):
        next_actions[:, i] = i
    
    
    # next_state_next_action
    tuplesMx0 = np.column_stack((train, next_actions[:,0]))
    tuplesMx1 = np.column_stack((train, next_actions[:,1]))
    tuplesMx2 = np.column_stack((train, next_actions[:,2]))
    tuplesMx3 = np.column_stack((train, next_actions[:,3]))
    tuplesMx4 = np.column_stack((train, next_actions[:,4]))
    tuplesMx5 = np.column_stack((train, next_actions[:,5]))
    tuplesMx6 = np.column_stack((train, next_actions[:,6]))
    tuplesMx7 = np.column_stack((train, next_actions[:,7]))
    tuplesMx8 = np.column_stack((train, next_actions[:,8]))
    tuplesMx9 = np.column_stack((train, next_actions[:,9]))
    tuplesMx10 = np.column_stack((train, next_actions[:,10]))
    tuplesMx11 = np.column_stack((train, next_actions[:,11]))
    
    return tuplesMx0, tuplesMx1, tuplesMx2, tuplesMx3, tuplesMx4, tuplesMx5, tuplesMx6, tuplesMx7, tuplesMx8, tuplesMx9, tuplesMx10, tuplesMx11

In [43]:
# tuplesMx0, tuplesMx1, tuplesMx2, tuplesMx3, tuplesMx4, tuplesMx5, tuplesMx6, tuplesMx7, tuplesMx8, tuplesMx9, tuplesMx10, tuplesMx11  = tuple_()
# tuplesMx0, tuplesMx1 = tuple_()

## Regression phase:

Before performing the prediction task .. let's split the data to training and validation sets .. 

To avoid any problems in prediction by having string variables .. let's binarize (catergorize) all the variables .. 

Guidance ==> https://pythonprogramming.net/rnn-tensorflow-python-machine-learning-tutorial/

## Multilayer Deep Neural Network (DQN):

https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/5_Deep_Q_Network/RL_brain.py

## Model Training:

https://stackoverflow.com/questions/46832151/tensorflow-neural-network-multi-layer-perceptron-for-regression-example

Guidance: https://www.youtube.com/watch?v=FbJw8J0rTyQ

In [45]:
def DQN_train(x_inputs, y_outputs):
    
    print np.shape(y_outputs)
    
    # Placeholder
    X = tf.placeholder(dtype=tf.float32, shape=[None, np.shape(x_inputs)[1] ])
    Y = tf.placeholder(dtype=tf.float32, shape=[None])


    # Model architecture parameters
    n_stocks = np.shape(x_inputs)[1] 
    n_neurons_1 = 40
    n_neurons_2 = 15
#     n_neurons_3 = 256
#     n_neurons_4 = 128
    n_target = 50 #np.shape(x_inputs)[0]
    
    batch_size = 500 
    epochs = 100
    
        # Initializers
    sigma = 1
    weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
    bias_initializer = tf.zeros_initializer()
    

    # Layer 1: Variables for hidden weights and biases
    W_hidden_1 = tf.Variable(weight_initializer([n_stocks, n_neurons_1]))
    bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))

    # Layer 2: Variables for hidden weights and biases
    W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
    bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))

    # Output layer: Variables for output weights and biases
    W_out = tf.Variable(weight_initializer([n_neurons_2, n_target]))
    bias_out = tf.Variable(bias_initializer([n_target]))


    # Hidden layer
    hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
    hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
#     hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
#     hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

    # Output layer (must be transposed)
    y_ = tf.add(tf.matmul(hidden_2, W_out), bias_out)
    
    # initialize variables
    init_op = tf.global_variables_initializer()
    
    # Cost function
    mse = tf.reduce_mean(tf.squared_difference(y_, Y))

    # Optimizer
    opt = tf.train.AdamOptimizer().minimize(mse)

    with tf.Session() as sess:
        sess.run(init_op)
        num_itr = int(np.shape(y_outputs)[0] / batch_size)
        
        for epoch in range(epochs):
            for i in range(num_itr):
                predicted_output = sess.run(y_, feed_dict={X: x_inputs, Y: y_outputs})
                
#                 _, c = sess.run([optimizer, mse], feed_dict={x: x_inputs, y: y_outputs})
    return predicted_output[:, 1]

Q(s,a) representing the (Quality) of action a at state is .. 

this Q value depends on the immediate reward r .. however, it'll be more effective if it takes the future rewards Q(s', a') into consideration .. 

the future rewards are discounted by probability gama .. cause the evironment is stochastic hence, it is uncertain that each time you select action a you gonna get the same reward r

In [48]:
def Q_learning():
    
    avg_Q = []  
    gamma = 0.9
#     actions = actions_prep(df)

# nepisode > https://stats.stackexchange.com/questions/250943/what-is-the-difference-between-episode-and-epoch-in-deep-q-learning
# means one complete path from state, action, next_s, next_a, reward upuntil terminal state

    nepisod = 1#2 #np.shape(actions)[1]  ##22+ 
    
    tuplesMx0, tuplesMx1, tuplesMx2, tuplesMx3, tuplesMx4, tuplesMx5, tuplesMx6, tuplesMx7, tuplesMx8, tuplesMx9, tuplesMx10, tuplesMx11 = tuple_()  # nepisod ==> 46, tuplesMx[:,-1] ==> 499   
    num_rows = np.shape(tuplesMx0)[0] #-1
    
    Q_predicted = np.zeros([num_rows, 12]) #num_rows, num_columns
    
    best_action = np.zeros([12, 1])
    
    
    for i in xrange(nepisod):
        Q_optimal = tuplesMx0[:,-2] + gamma*np.amax(Q_predicted, axis=1)  #np.max(Q_predicted) # returns max value per row !
        Q_predicted0 = DQN_train(tuplesMx0, Q_optimal)
        Q_predicted1 = DQN_train(tuplesMx1, Q_optimal)
        Q_predicted2 = DQN_train(tuplesMx2, Q_optimal)
        Q_predicted3 = DQN_train(tuplesMx3, Q_optimal)
        Q_predicted4 = DQN_train(tuplesMx4, Q_optimal)
        Q_predicted5 = DQN_train(tuplesMx5, Q_optimal)
        Q_predicted6 = DQN_train(tuplesMx6, Q_optimal)
        Q_predicted7 = DQN_train(tuplesMx7, Q_optimal)
        Q_predicted8 = DQN_train(tuplesMx8, Q_optimal)
        Q_predicted9 = DQN_train(tuplesMx9, Q_optimal)
        Q_predicted10 = DQN_train(tuplesMx10, Q_optimal)
        Q_predicted11 = DQN_train(tuplesMx11, Q_optimal)
        Q_predicted = np.column_stack((Q_predicted0, Q_predicted1, Q_predicted2, Q_predicted3,
                                      Q_predicted4, Q_predicted5, Q_predicted6, Q_predicted7,
                                      Q_predicted8, Q_predicted9, Q_predicted10, Q_predicted11))
    
#     for i in xrange(num_rows):
    best_action = np.argmax(Q_predicted, axis=1) 
        
    return best_action

In [49]:
best_action = Q_learning()
best_action

(209908,)
(209908,)
(209908,)
(209908,)
(209908,)
(209908,)
(209908,)
(209908,)
(209908,)
(209908,)
(209908,)
(209908,)


array([ 8,  7,  7, ...,  7,  9, 10])

In [64]:
# Counter(best_action)

unique, counts = np.unique(best_action, return_counts=True)

best_actions_list = np.asarray((unique, counts)).T

print "Freq."
print best_actions_list[np.argsort(best_actions_list[:, 1])]

Freq.
[[    0     3]
 [    5    11]
 [    2   209]
 [    4  1000]
 [    9 15854]
 [   10 19221]
 [    8 28150]
 [    7 50851]
 [    3 94609]]


In [65]:
# print "Relative Freq."
# print np.asarray((unique, counts/np.float(len(best_action)))) #.T

# Comment:
Increasing batch size .. increases the output non zero values 